In [ ]:
import os
import pandas as pd
from pathlib import Path
from zipfile import ZipFile
years = []
basePath =Path("C:\\Users\sivar\OneDrive\code\oneminutedata")
for f in basePath.iterdir():
    if f.is_dir():
        if ("RAW" in str(f) or "Upto" in str(f)):
            x = 1
        else:
            years.append(Path(f))
        
print(years)

In [ ]:
monthlist = []
for year in years:
    if year.is_dir():
        for month in year.iterdir():
            if month.is_dir():
                monthlist.append((Path(month)))
print(monthlist)

In [ ]:
targetBasePath = "C:\\data\\Extracted1\\"

In [ ]:
for mt in monthlist:
    extractedPathStr = targetBasePath + mt.parent.name + "\\" + mt.name
    extractedPath = Path(extractedPathStr)
    for m in mt.iterdir():
        if(str(m).endswith(".zip")):
            with ZipFile(m,'r') as zipObj:
                zipObj.extractall(extractedPath)
        else:
            print(mt.parent.name)
            print(mt.name)


In [ ]:
import os
def run_fast_scandir(dir, ext):    # dir: str, ext: list
    subfolders, files = [], []

    for f in os.scandir(dir):
        if f.is_dir():
            subfolders.append(f.path)
            #print(subfolders)
        if f.is_file():
            if os.path.splitext(f.name)[1].lower() in ext:
                files.append(f.path)


    for dir in list(subfolders):
        sf, f = run_fast_scandir(dir, ext)
        subfolders.extend(sf)
        files.extend(f)
    return subfolders, files


subfolders, files1 = run_fast_scandir(targetBasePath, [".txt"])

In [ ]:
from pathlib import Path
symlist = []
for file in files1:
    symlist.append(Path(file).name)
symbols = list(dict.fromkeys(symlist))

In [ ]:
consolidatedpath = "c:\data\symbols"
for symbol in symbols:
    print(symbol)
    fileName = consolidatedpath+"\\"+symbol
    with open(Path(fileName), 'w') as output_file:
        for file in files1:
            if Path(file).name == symbol:
                with open(Path(file)) as input_file:
                    output_file.write(input_file.read())

### SPOT ROW EXAMPLE

BANKNIFTY,20121031,15:30,11246.50,11251.80,11243.95,11250.20

BANKNIFTY,20160901,09:16,19804.05,19804.05,19771.00,19784.95,54,0

### FUTURES ROW EXAMPLE

BANKNIFTY_F1,20121031,15:30,11326.00,11334.00,11320.05,11334.00,17500
BANKNIFTY_F1,20160901,09:16,19879.85,19879.85,19849.75,19854.00,33920,2825760

### SPOT COLUMNS
Symbol, Date, Time, Open, High, Log. Close, Volume, Value

### Futures Columns
Symbol, Date, Time, Open, High, Low, Close, Volume, Value

In [ ]:
import pandas as pd
symbolpath = "c:\data\symbols"
colnames=['Symbol', 'Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Value'] 
df = pd.read_csv("c:\data\symbols\BANKNIFTY_F1.txt", names=colnames)
df['Date'] = df['Date'].astype(str)
df['Date'] = df['Date'].str[:8]
df['Date'] = df['Date'] + ' ' + df['Time']
df['Date'] = pd.to_datetime(df['Date'], format = "%Y%m%d %H:%M")
df.set_index('Date', inplace=True)
ohlc = {
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum',
    'Value' : 'sum'
}

df5min = df.resample('5min', base=15).apply(ohlc)
df5min.head()
df5min = df5min.between_time('09:15', '15:30')
df5min = df5min.drop(df5min[df5min.Open.isnull()].index)

In [ ]:
df5min.tail()
